In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("dados/acidentes_naofatais.csv", encoding = "latin", sep = ";")

In [83]:
acidentes = (df
.rename({
    "Data do Acidente": "data",
    "Logradouro": "rua",
    "Município": "nm_municipio",
    "Veículos Envolvidos - Motocicleta": "motocicletas",
    "Pessoas Envolvidas - Leve": "leve",
    "Pessoas Envolvidas - Grave": "grave",
}, axis = 1)
.filter(["data", "nm_municipio", "rua", "grave", "leve", "motocicletas"])
.query(f"(grave > 0 or leve > 0) and motocicletas > 0 and nm_municipio == 'SAO PAULO'")
.assign(
    mes = lambda _: _.data.str[5:7], 
    ano = lambda _: _.data.str[0:4])
)
acidentes.head()

,data,nm_municipio,rua,grave,leve,motocicletas,mes,ano
44,2023-09-30,SAO PAULO,RUA DAS FLECHAS,0,1,1,09,2023
50,2023-09-30,SAO PAULO,AVENIDA SANTOS DUMONT,0,1,1,09,2023
51,2023-09-30,SAO PAULO,AVENIDA RAIMUNDO PEREIRA DE MAGALHAES,0,1,1,09,2023
56,2023-09-30,SAO PAULO,AVENIDA JACUPESSEGO,0,1,1,09,2023
69,2023-09-30,SAO PAULO,SP 015,0,2,1,09,2023


In [84]:
avenidas = (acidentes
.groupby(["ano", "rua"])
.aggregate({"motocicletas": "count"})
.reset_index()
.sort_values(by = ["ano", "motocicletas"])
# .query("rua.str.contains('BANDEIRANTES')")
.query("ano == '2021' and ~ rua.str.contains('SP ') and ~ rua.str.contains('SPA ') and ~ rua.str.contains('BR ')")
.tail(50)
)
avenidas.tail()

,ano,rua,motocicletas
8953,2021,AVENIDA WASHINGTON LUIS,147
8903,2021,AVENIDA SAPOPEMBA,157
8159,2021,AVENIDA ARICANDUVA,159
9013,2021,ESTRADA DE ITAPECERICA,174
8912,2021,AVENIDA SENADOR TEOTONIO VILELA,175


In [92]:
(acidentes
 .query(f"rua in {avenidas.rua.to_list()}")
 .groupby(["ano", "mes", "rua"])
 .aggregate({"motocicletas": "count"})
 .rename({"motocicletas": "acidentes"}, axis = 1)
 .reset_index()
)

,ano,mes,rua,acidentes
0,2019,01,AVENIDA ALCANTARA MACHADO,2
1,2019,01,AVENIDA ARICANDUVA,8
2,2019,01,AVENIDA ATLANTICA,2
3,2019,01,AVENIDA BRIGADEIRO LUIS ANTONIO,1
4,2019,01,AVENIDA CARLOS LACERDA,4
...,...,...,...,...
2731,2023,09,ESTRADA DO CAMPO LIMPO,9
2732,2023,09,ESTRADA DO M'BOI MIRIM,16
2733,2023,09,NAO DISPONIVEL,3
2734,2023,09,RUA SADAMU INOUE,2
